The **[fundamental matrix](https://en.wikipedia.org/wiki/Fundamental_matrix_(computer_vision))** $F$ is a matrix that maps points in one image to their matches in the other image.
Ok, technically it doesn't match points to points, but points to their respective epipolar lines.
Due to the [epipolar constraint](https://en.wikipedia.org/wiki/Epipolar_geometry#Epipolar_constraint_and_triangulation), it is guaranteed that a point in one image plane will have its match in the other (stereo) image frame on a line (called the epipolar line).
This is really cool, because now when we search for a match, we just need to search a line in the image instead of the whole 2-D image.
This constraint restricts our search space by a lot!
Here, we will use this fact in a reverse way.
We already have the matches, thanks to our brute-force matcher -- albeit not all of them are high quality as we saw above.
And we know that the matches must satisfy the epipolar constraint, which is basically saying that they must follow the mapping (or transformation) of the fundamental matrix.
Now, we can tell by looking at each match if it is a good quality match or not, but this constraint gives us a quantitative way of deciding whether a match is a good match or not.
So, the points that don't obey this fundamental matrix transform (or mapping) are bad matches.

There is an algorithm that makes this check for us, called [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus).
We will use this to filter out bad matches.

In [1]:
matches1 = np.asarray([kp1[m.queryIdx].pt for m in matches])
matches2 = np.asarray([kp1[m.trainIdx].pt for m in matches])

F, inlierMask = cv2.findFundamentalMat(matches1, matches2, cv2.RANSAC, 3, 0.9, 100)
print("Number of inliers: ", sum(inlierMask))

NameError: name 'np' is not defined

### Motion estimation

We want to figure out how the camera moved from frame 1 to frame 2.
This is a geometry problem.
Since each frame is a 2-D projection of 3-D physical space, the mathematical field of [projective geometry](https://www.britannica.com/science/projective-geometry) comes in handy.
To be honest, I don't fully understand projective geometry the way I do regular (Euclidean) geometry, but it doesn't seem entirely necessary for this particular application.
I will definitely be learning that more deeply as time goes on, but [here](https://www.youtube.com/playlist?list=PL2zRqk16wsdoCCLpou-dGo7QQNks1Ppzo) is an interim explanation that should prove useful (yes, the whole playlist).

### References

* [First Principles of Computer Vision](https://www.youtube.com/channel/UCf0WB91t8Ky6AuYcQV0CcLw) YouTube channel
* [twitchslam](https://github.com/geohot/twitchslam) by George Hotz
* [KITTI Odometry with OpenCV Python](https://www.youtube.com/playlist?list=PLrHDCRerOaI9HfgZDbiEncG5dx7S3Nz6X) by Nate Cibik
* Visual Odometry by David Scaramuzza and Friedrich Fraundorfer